In [1]:
import Bio.SeqIO as SeqIO
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import os
import sys

In [2]:
def calc_aai(seq1, seq2, gap_rate=0.8):
    if len(seq1) != len(seq2):
        raise Exception('Sequence not aligned properly!')
    else:
        seq1_np = np.array(list(seq1))
        seq2_np = np.array(list(seq2))

        valid_positions = ~((seq1_np == '-') & (seq2_np == '-') & (seq1_np == seq2_np))
        match_count = np.sum(seq1_np[valid_positions] == seq2_np[valid_positions])
        gap_count = np.sum((seq1_np[valid_positions] == '-') | (seq2_np[valid_positions] == '-'))
        aligned_length = np.sum(valid_positions)
        
        try:
            pident = match_count / (aligned_length) * 100
        except ZeroDivisionError:
            pident = 'NA'
        
        return match_count, (aligned_length - match_count), gap_count, aligned_length, len(seq1), pident

In [3]:
assembly_seqs = open('assembly_contigid.list').read().splitlines()
assembly_seqs

['AstV-43/HD13634',
 'SpinV-2/HD13634',
 'ParV-1/HL08',
 'PicoV-1/HL09',
 'AstV-14/HL13',
 'ParV-2/HD13300',
 'AstV-35/HD13564',
 'AstV-39/HD13305',
 'CoV-5/HD13308',
 'CoV-6/HD13313',
 'AstV-40/HD13316',
 'AstV-41/HD13328',
 'AstV-42/HD13329',
 'PicoV-16/HD13336',
 'PicoV-17/HD13337',
 'PicoV-18/HD13337',
 'CoV-7/HD13339',
 'CalV-5/HD13355',
 'CV-8/HD13578',
 'PicoV-19/HD13580',
 'PicoV-20/HD13580',
 'PicoV-21/HD13599',
 'AstV-55/HD13620',
 'AstV-56/HD13620',
 'AstV-57/HD13624',
 'AstV-58/HD13625',
 'CoV-9/HD13625',
 'CV-3/HD13630',
 'AdV-1/HD13637',
 'CV-4/LH12152',
 'AdV-3/LH12168',
 'AstV-28/LH12171',
 'AstV-20/LH12171',
 'AstV-26/LH12171',
 'AstV-30/LH12171',
 'AstV-22/LH12171',
 'AstV-23/LH12171',
 'AstV-24/LH12171',
 'AstV-25/LH12171',
 'AstV-27/LH12171',
 'AstV-29/LH12171',
 'AstV-21/LH12171',
 'CoV-2/LH12177',
 'PicoV-9/LH12177',
 'PicoV-11/LH12181',
 'PicoV-10/LH12181',
 'CalV-2/LH12181',
 'AstV-31/LH12181',
 'CV-5/10',
 'AstV-32/10',
 'AstV-33/12',
 'SpinV-1/15',
 'AstV-44/1

In [4]:
result = []
for file in os.listdir("/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/HostSwitching/4.rmdup"):
    msafile = os.path.join("/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/HostSwitching/4.rmdup", file)
    vf = file.split('_')[1]
    aln = SeqIO.to_dict(list(SeqIO.parse(msafile, format='fasta')))
    aln_cleaned = { seqid: ''.join([ char for char in list(seqrecord.seq) if (char == '-') or (char.isupper()) ]) for seqid, seqrecord in aln.items() }
    aln_cleaned_seqids = aln_cleaned.keys()
    aln_cleaned_asemseqs = set(aln_cleaned_seqids) & set(assembly_seqs)
    #aln_cleaned_pubseqs = set(aln_cleaned_seqids) - set(assembly_seqs)
    for seqid_i in tqdm(aln_cleaned_asemseqs):
        for seqid_j in aln_cleaned_seqids:
            seq_i = aln_cleaned[seqid_i]
            seq_j = aln_cleaned[seqid_j]
            match, mismatch, gap, alnlen, msalen, pident = calc_aai(seq_i, seq_j, gap_rate=0.8)
            result.append([vf, seqid_i, seqid_j, match, mismatch, gap, alnlen, msalen, pident])

100%|██████████| 1/1 [00:00<00:00,  5.40it/s]


In [5]:
aai_resultdf = pd.DataFrame(result, columns=['viralfamily', 'assembled_seq', 'sseq', 'match', 'mismatch', 'gap', 'alnlen', 'msalen', 'pident'])
aai_resultdf

,viralfamily,assembled_seq,sseq,match,mismatch,gap,alnlen,msalen,pident
0,AstV,AstV-48/451,YP_003090286.1,121,87,0,208,208,58.173077
1,AstV,AstV-48/451,CAB95006.3,107,101,0,208,208,51.442308
2,AstV,AstV-48/451,YP_006905856.1,115,93,0,208,208,55.288462
3,AstV,AstV-48/451,YP_006905859.1,114,94,0,208,208,54.807692
4,AstV,AstV-48/451,YP_003275951.1,117,91,0,208,208,56.250000
...,...,...,...,...,...,...,...,...,...
216562,HerpV,HerpV-1/CH191,NC_055142.1_96,143,326,205,469,476,30.490405
216563,HerpV,HerpV-1/CH191,NC_076963.1_112,143,326,205,469,476,30.490405
216564,HerpV,HerpV-1/CH191,OM517184.1_17,138,331,205,469,476,29.424307
216565,HerpV,HerpV-1/CH191,OP934214.1_7,142,327,205,469,476,30.277186


In [7]:
aai_filtered = aai_resultdf[(aai_resultdf['pident'] >= 75) & (aai_resultdf['gap']< 100) & (aai_resultdf['alnlen'] > 50) & (aai_resultdf['assembled_seq']!=aai_resultdf['sseq'])]
aai_filtered

,viralfamily,assembled_seq,sseq,match,mismatch,gap,alnlen,msalen,pident
49,AstV,AstV-48/451,ACF75854.1,159,49,0,208,208,76.442308
50,AstV,AstV-48/451,ACN88713.1,160,48,0,208,208,76.923077
53,AstV,AstV-48/451,QVW10123.1,159,49,0,208,208,76.442308
63,AstV,AstV-48/451,AstV-10/210524,160,48,0,208,208,76.923077
64,AstV,AstV-48/451,AstV-9/210524,156,52,0,208,208,75.000000
...,...,...,...,...,...,...,...,...,...
215792,HepV,HepV-2/B86,MW249013.1_1,122,33,0,155,156,78.709677
215849,HepV,HepV-2/B86,NC_018382.1_2,122,33,0,155,156,78.709677
215852,HepV,HepV-2/B86,NC_076569.1_1,127,28,0,155,156,81.935484
215855,HepV,HepV-2/B86,NC_076918.1_1,119,36,0,155,156,76.774194


In [8]:
suffix_lines = aai_filtered['sseq'].apply(lambda x: bool(re.search(r'.*_\d$', x)))
aai_filtered.loc[suffix_lines, 'sseq'] = aai_filtered.loc[suffix_lines, 'sseq'].apply(lambda x: '_'.join(x.split('_')[:-1]))

In [9]:
virioninfo_human = pd.read_table('/jdfssz1/ST_HEALTH/P20Z10200N0206/wangdaxi/Projects/SouthChinaBat/host_switching/RepViralSequences_133seqs_human.blast.add', skiprows=1, header=None, 
                           names='qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore host_species virusname host_species2'.split())
virioninfo_nonhuman = pd.read_table('/jdfssz1/ST_HEALTH/P20Z10200N0206/wangdaxi/Projects/SouthChinaBat/host_switching/RepViralSequences_133seqs_nonhuman.blast.add', skiprows=1, header=None, 
                           names='qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore host_species virusname host_species2'.split())
virioninfo_full = pd.concat([virioninfo_human, virioninfo_nonhuman]).dropna(subset='host_species')
virioninfo_full['host_species'] = virioninfo_full['host_species'].apply(lambda x: x.capitalize())

In [10]:
aai_filtered_pub = aai_filtered[~aai_filtered['sseq'].isin(assembly_seqs)]
aai_filtered_pub = aai_filtered_pub.merge(virioninfo_full[['sseqid', 'host_species', 'virusname']].drop_duplicates(), left_on='sseq', right_on='sseqid')
aai_filtered_pub

,viralfamily,assembled_seq,sseq,match,mismatch,gap,alnlen,msalen,pident,sseqid,host_species,virusname
0,AstV,AstV-48/451,EU847146.1,159,49,0,208,208,76.442308,EU847146.1,Myotis chinensis,bat astrovirus 1
1,AstV,AstV-44/17,EU847146.1,170,38,0,208,208,81.730769,EU847146.1,Myotis chinensis,bat astrovirus 1
2,AstV,AstV-18/SZ147,EU847146.1,173,35,0,208,208,83.173077,EU847146.1,Myotis chinensis,bat astrovirus 1
3,AstV,AstV-19/XL26,EU847146.1,172,36,0,208,208,82.692308,EU847146.1,Myotis chinensis,bat astrovirus 1
4,AstV,AstV-50/161,EU847146.1,170,38,0,208,208,81.730769,EU847146.1,Myotis chinensis,bat astrovirus 1
...,...,...,...,...,...,...,...,...,...,...,...,...
15653,HepV,HepV-2/B86,MW249013.1,122,33,0,155,156,78.709677,MW249013.1,Desmodus rotundus,chirohepevirus eptesici
15654,HepV,HepV-2/B86,NC_018382.1,122,33,0,155,156,78.709677,NC_018382.1,Eptesicus serotinus,chirohepevirus eptesici
15655,HepV,HepV-2/B86,NC_076569.1,127,28,0,155,156,81.935484,NC_076569.1,Rhinolophus ferrumequinum,chirohepevirus eptesici
15656,HepV,HepV-2/B86,NC_076918.1,119,36,0,155,156,76.774194,NC_076918.1,Desmodus rotundus,chirohepevirus eptesici


In [11]:
requant_info = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/Projects/GIZ_SouthernChinaBat_clean/HostSwitching/TableS3.GIZ_SouthernChinaBats_VirusQuant_new_rpm0.5_20231027.xlsx')
requant_info_use = requant_info[['qseqid', 'Host_species']].rename(columns={"qseqid":"sseq", "Host_species":"host_species"}).drop_duplicates()
requant_info_use['virusname'] = requant_info_use["sseq"]
requant_info_use['host_genus'] = requant_info_use["host_species"].apply(lambda x: x.split()[0])
requant_info_use

,sseq,host_species,virusname,host_genus
0,AstV-43/HD13634,Rousettus leschenaultii,AstV-43/HD13634,Rousettus
1,SpinV-2/HD13634,Rousettus leschenaultii,SpinV-2/HD13634,Rousettus
4,AstV-14/HL13,Pipistrellus abramus,AstV-14/HL13,Pipistrellus
5,PicoV-1/HL09,Pipistrellus abramus,PicoV-1/HL09,Pipistrellus
6,ParV-1/HL08,Pipistrellus abramus,ParV-1/HL08,Pipistrellus
...,...,...,...,...
573,RhaV-2/210537,Rhinolophus affinis,RhaV-2/210537,Rhinolophus
575,HepV-1/SZ111,Rhinolophus sinicus,HepV-1/SZ111,Rhinolophus
579,ParV-14/CH035,Rhinolophus affinis,ParV-14/CH035,Rhinolophus
593,PicoV-7/CX08,Rhinolophus sinicus,PicoV-7/CX08,Rhinolophus


In [12]:
requant_info['Host_genus'] = requant_info["Host_species"].apply(lambda x: x.split()[0])
rhinolophus_virus = list(requant_info.loc[requant_info['Host_genus']=='Rhinolophus', 'qseqid'])
rhinolophus_virus

['CoV-3/B95',
 'AstV-35/HD13564',
 'AstV-35/HD13564',
 'ParV-2/HD13300',
 'ParV-9/CH230',
 'AdV-2/SZ099',
 'PicoV-19/HD13580',
 'AstV-35/HD13564',
 'PicoV-2/CH085',
 'AstV-6/CH221',
 'PicoV-4/CH204',
 'AstV-35/HD13564',
 'HepV-2/B86',
 'AstV-3/CH200',
 'AstV-4/CH203',
 'AstV-18/SZ147',
 'AstV-38/CH007',
 'AstV-17/SZ141',
 'AstV-18/SZ147',
 'PicoV-24/201320',
 'CoV-4/CH053',
 'AdV-4/200631',
 'ParV-12/200635',
 'AstV-54/201311',
 'CalV-4/201335',
 'AstV-1/CH179',
 'CV-8/HD13578',
 'AstV-35/HD13564',
 'PicoV-19/HD13580',
 'PicoV-20/HD13580',
 'CoV-8/200604',
 'AdV-2/SZ099',
 'PicoV-19/HD13580',
 'PicoV-2/CH085',
 'CoV-4/CH053',
 'PicoV-4/CH204',
 'PicoV-5/CH221',
 'CoV-8/200604',
 'AstV-54/201311',
 'AstV-1/CH179',
 'CoV-3/B95',
 'CoV-8/200604',
 'PicoV-21/HD13599',
 'AstV-34/B80',
 'AstV-37/B103',
 'AstV-1/CH179',
 'AstV-3/CH200',
 'AstV-5/CH203',
 'PicoV-4/CH204',
 'AstV-6/CH221',
 'AstV-7/CH221',
 'AstV-9/210524',
 'AstV-10/210524',
 'AstV-11/210524',
 'AstV-16/SZ123',
 'AstV-17/SZ141

In [13]:
aai_filtered_asem = aai_filtered[aai_filtered['sseq'].isin(assembly_seqs)]
aai_filtered_asem = aai_filtered_asem.merge(requant_info_use.drop('host_genus', axis=1), on='sseq', how='left')
aai_filtered_asem

,viralfamily,assembled_seq,sseq,match,mismatch,gap,alnlen,msalen,pident,host_species,virusname
0,AstV,AstV-48/451,AstV-10/210524,160,48,0,208,208,76.923077,Rhinolophus affinis,AstV-10/210524
1,AstV,AstV-48/451,AstV-9/210524,156,52,0,208,208,75.000000,Rhinolophus affinis,AstV-9/210524
2,AstV,AstV-48/451,AstV-34/B80,158,50,0,208,208,75.961538,Rhinolophus affinis,AstV-34/B80
3,AstV,AstV-48/451,AstV-34/B80,158,50,0,208,208,75.961538,Rhinolophus sinicus,AstV-34/B80
4,AstV,AstV-48/451,AstV-38/CH007,161,47,0,208,208,77.403846,Rhinolophus sinicus,AstV-38/CH007
...,...,...,...,...,...,...,...,...,...,...,...
822,PicoV,PicoV-11/LH12181,PicoV-10/LH12181,168,4,0,172,176,97.674419,Miniopterus fuliginosus,PicoV-10/LH12181
823,PicoV,PicoV-11/LH12181,PicoV-10/LH12181,168,4,0,172,176,97.674419,Myotis ricketti,PicoV-10/LH12181
824,PicoV,PicoV-17/HD13337,PicoV-16/HD13336,141,34,1,175,176,80.571429,Miniopterus pusillus,PicoV-16/HD13336
825,PicoV,PicoV-17/HD13337,PicoV-16/HD13336,141,34,1,175,176,80.571429,Miniopterus fuliginosus,PicoV-16/HD13336


In [14]:
aai_mergeinfo_full = pd.concat([aai_filtered_asem, aai_filtered_pub.drop('sseqid', axis=1)]).sort_values(['assembled_seq', 'pident'], ascending=[True, False])
aai_mergeinfo_full['host_genus'] = aai_mergeinfo_full['host_species'].apply(lambda x: x.split()[0])
aai_mergeinfo_full = aai_mergeinfo_full.merge(requant_info_use[['sseq', 'host_genus']].rename(columns={"sseq": "assembled_seq", "host_genus":"requant_host_genus"}))
aai_mergeinfo_full

,viralfamily,assembled_seq,sseq,match,mismatch,gap,alnlen,msalen,pident,host_species,virusname,host_genus,requant_host_genus
0,AstV,AstV-1/CH179,AstV-28/LH12171,113,19,0,132,208,85.606061,Miniopterus pusillus,AstV-28/LH12171,Miniopterus,Rhinolophus
1,AstV,AstV-10/210524,AstV-16/SZ123,189,19,0,208,208,90.865385,Rhinolophus affinis,AstV-16/SZ123,Rhinolophus,Rhinolophus
2,AstV,AstV-10/210524,AstV-18/SZ147,189,19,0,208,208,90.865385,Rhinolophus affinis,AstV-18/SZ147,Rhinolophus,Rhinolophus
3,AstV,AstV-10/210524,AstV-18/SZ147,189,19,0,208,208,90.865385,Rhinolophus sinicus,AstV-18/SZ147,Rhinolophus,Rhinolophus
4,AstV,AstV-10/210524,AstV-38/CH007,185,23,0,208,208,88.942308,Rhinolophus sinicus,AstV-38/CH007,Rhinolophus,Rhinolophus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20703,RhaV,RhaV-2/210537,MW557338.1,143,27,1,170,170,84.117647,Rhinolophus ferrumequinum,vesiculovirus mediterranean,Rhinolophus,Rhinolophus
20704,RhaV,RhaV-2/210537,NC_076842.1,143,27,1,170,170,84.117647,Rhinolophus sinicus,vesiculovirus yinshui,Rhinolophus,Rhinolophus
20705,RhaV,RhaV-2/210537,NC_076939.1,143,27,1,170,170,84.117647,Rhinolophus ferrumequinum,vesiculovirus mediterranean,Rhinolophus,Rhinolophus
20706,RhaV,RhaV-2/210537,NC_077173.1,132,38,1,170,170,77.647059,Myotis altarium,wufeng myotis altarium vesiculovirus 1,Myotis,Rhinolophus


In [15]:
aai_mergeinfo_full_rhinolophus = aai_mergeinfo_full#[aai_mergeinfo_full['assembled_seq'].isin(rhinolophus_virus)]
aai_summary = []
for (asem_seqid, requant_host_genus), aaidf in aai_mergeinfo_full_rhinolophus.groupby(['assembled_seq', 'requant_host_genus']):
    try:
        samegenus_nearest = aaidf[aaidf['host_genus']==aaidf['requant_host_genus']].reset_index(drop=True).iloc[0,:]
        sg_record = [samegenus_nearest['sseq'], samegenus_nearest['pident'], samegenus_nearest['host_genus'], samegenus_nearest['virusname']]
    except:
        sg_record = ['NA' for i in range(4)]
    try:
        diffgenus_nearest = aaidf[aaidf['host_genus']!=aaidf['requant_host_genus']].reset_index(drop=True).iloc[0,:]
        dg_record = [diffgenus_nearest['sseq'], diffgenus_nearest['pident'], diffgenus_nearest['host_genus'], diffgenus_nearest['virusname']]
    except:
        dg_record = ['NA' for i in range(4)]
        
    aai_summary.append([asem_seqid, requant_host_genus] + sg_record + dg_record)

In [17]:
aai_summary = pd.DataFrame(aai_summary, columns=['seqid', 'requant_hostgenus', 'nearest_sg_seqid', 'nearest_sg_pident', 'nearest_sg_genus', 'nearest_sg_vname',
                                   'nearest_dg_seqid', 'nearest_dg_pident', 'nearest_dg_genus', 'nearest_dg_vsname'])
aai_summary['viralfamily'] = aai_summary['seqid'].apply(lambda x: x.split('-')[0])
aai_summary.to_excel('Assembly_toVIRION_AAI_summary.xlsx', index=False)

In [20]:
aai_filtered.merge(virioninfo_full[['sseqid', 'host_species', 'virusname']].drop_duplicates(), left_on='sseq', right_on='sseqid').sort_values('pident', ascending=False).drop_duplicates(['assembled_seq', 'host_species', 'virusname']).dropna().to_excel('Assembled_Sequence_toVirion_AAI_v2.xlsx', index=False)